In [3]:
import os
import zipfile  # untuk membuat file ZIP
import fitz  # PyMuPDF untuk membaca PDF
from docx import Document  # Untuk membaca file DOCX
import tiktoken  # Untuk menghitung token
import io
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv


In [ ]:
import os
import zipfile  # untuk membuat file ZIP
import fitz  # PyMuPDF untuk membaca PDF
from docx import Document  # Untuk membaca file DOCX
import tiktoken  # Untuk menghitung token
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

# Memuat API Key dari .env
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Direktori penyimpanan FAISS Index
FAISS_PATH = "Output_Faiss"

# Fungsi membaca PDF
def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text() + "\n"
    return text

# Fungsi membaca DOCX
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Fungsi menghitung token
def count_tokens(text, model="text-embedding-ada-002"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Fungsi untuk membuat folder dengan nama unik (faiss_index1, faiss_index2, dst)
def create_unique_folder(path=FAISS_PATH):
    # Pastikan direktori FAISS_PATH ada
    if not os.path.exists(path):
        os.makedirs(path)

    # Cek folder dengan nama faiss_indexX yang tersedia
    i = 1
    while os.path.exists(f"{path}/faiss_index{i}"):
        i += 1
    # Buat folder baru dengan nama faiss_indexX yang tidak ada
    unique_folder = f"{path}/faiss_index{i}"
    os.makedirs(unique_folder)  # Pastikan folder dibuat
    return unique_folder

# Fungsi menyimpan FAISS ke file dan menyimpan daftar nama file dalam notepad
def save_faiss_index(vector_store, file_list, combined_text, path=FAISS_PATH):
    # Membuat folder baru dengan nama unik
    index_folder = create_unique_folder(path)

    # Simpan FAISS index
    vector_store.save_local(index_folder)
    print(f"✅ FAISS index telah disimpan di: {index_folder}")

    # Simpan daftar file yang digunakan ke dalam file_list.txt
    file_list_path = os.path.join(index_folder, "file_list.txt")
    with open(file_list_path, "w") as f:
        for file_name in file_list:
            f.write(file_name + "\n")
    # Simpan combined_text ke dalam file combine_text.txt
    file_list_path = os.path.join(index_folder, "combine_text.txt")

    with open(file_list_path, "w") as f:
        f.write(combined_text)  # Simpan seluruh combined_text ke dalam file
    print(f"✅ Daftar file telah disimpan di: {file_list_path}")

# Fungsi untuk memproses file yang di-upload
def process_files(input_path, path=FAISS_PATH):
    combined_text = ""
    file_list = []

    # Membaca file dari folder yang diberikan di input_path
    if not os.path.exists(input_path):
        raise ValueError(f"Direktori atau file tidak ditemukan: {input_path}")
    
    # Jika input adalah direktori
    if os.path.isdir(input_path):
        uploaded_files = []
        for file_name in os.listdir(input_path):
            file_path = os.path.join(input_path, file_name)
            if file_name.endswith(".pdf"):
                uploaded_files.append(file_path)
        
        # Proses file berdasarkan ekstensi
        for file_path in uploaded_files:
            file_name = os.path.basename(file_path)
            file_list.append(file_name)
            combined_text += read_pdf(file_path) + "\n"
    else:
        raise ValueError("Pastikan path yang diberikan adalah folder.")

    # Memecah teks menjadi chunks untuk FAISS
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_text(combined_text)

    # Menghitung total token
    total_tokens = sum(count_tokens(chunk) for chunk in chunks)
    print(f"Total token yang digunakan: {total_tokens}")

    # Membuat embedding dan FAISS
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)

    # Simpan FAISS index
    save_faiss_index(vector_store, file_list, combined_text)

# Path file/folder yang ingin diproses
input_folder_path = r'D:\01. Bapak\Github\HazChat\Temp\Test\membuat_faiss_file\Input_file copy'

# Memproses file dari folder
process_files(input_folder_path)


Total token yang digunakan: 923
✅ FAISS index telah disimpan di: Output_Faiss/faiss_index4
✅ Daftar file telah disimpan di: Output_Faiss/faiss_index4\file_list.txt
